In [1]:
import pandas as pd

from sklearn.preprocessing import RobustScaler
from catboost import CatBoostClassifier

Загрузим обучающую и тестовую выборку из первой части проекта.

In [2]:
train = pd.read_csv('/kaggle/input/data-distance/train_distance.csv')

In [3]:
train.head()

,offer_depersanalised,goods_depersanalised,sum_length,attrs+title_score,offer_price,goods_price,goods_category_id,target,id,b,...,p,q,r,s,v,w,x,y,dist,dist_title
0,295140,1396793,37,0.027267,1070.0,NaN,14.0,0,295140$1396793,NaN,...,4.608245,2.984364,0.801767,1.404596,0.113886,0.059486,-0.192472,-0.089284,NaN,0.247251
1,65291,1396586,38,0.050415,698.0,NaN,14.0,0,65291$1396586,NaN,...,NaN,NaN,NaN,NaN,0.377817,0.018323,-0.202608,0.014667,NaN,0.388207
2,39232,1396244,38,0.087280,837.0,NaN,14.0,0,39232$1396244,NaN,...,NaN,NaN,NaN,NaN,0.133571,-0.049654,-0.091354,0.007452,NaN,0.323675
3,39232,1396513,38,0.087280,837.0,NaN,14.0,0,39232$1396513,NaN,...,NaN,NaN,NaN,NaN,0.133571,-0.049654,-0.091354,0.007452,NaN,0.225795
4,65052,1396237,38,0.079773,1085.0,NaN,14.0,0,65052$1396237,NaN,...,NaN,NaN,NaN,NaN,0.091601,0.053658,-0.188811,-0.075650,NaN,0.139631


In [4]:
test = pd.read_csv('/kaggle/input/data-distance/test_distance.csv')

In [5]:
test.head()

,offer_depersanalised,goods_depersanalised,sum_length,attrs+title_score,offer_price,goods_price,goods_category_id,id,b,c,...,p,q,r,s,v,w,x,y,dist,dist_title
0,64819,1396468,38,0.046997,368.0,NaN,14.0,64819$1396468,NaN,NaN,...,NaN,NaN,NaN,NaN,0.046283,-0.061824,0.119573,-0.018517,NaN,0.189330
1,64819,1396235,38,0.046997,368.0,NaN,14.0,64819$1396235,NaN,NaN,...,NaN,NaN,NaN,NaN,0.046283,-0.061824,0.119573,-0.018517,NaN,0.186856
2,64819,1396318,38,0.046997,368.0,NaN,14.0,64819$1396318,NaN,NaN,...,NaN,NaN,NaN,NaN,0.046283,-0.061824,0.119573,-0.018517,NaN,0.136870
3,359959,1396281,40,0.060211,634.0,NaN,14.0,359959$1396281,6.765518,3.602556,...,0.247481,1.806707,-2.586108,-0.441305,0.241693,-0.183809,0.030217,0.147143,10.193567,0.358583
4,142700,717657,40,0.000370,14924.0,31840.0,2.0,142700$717657,NaN,NaN,...,-1.484497,4.012441,-6.217536,6.051277,0.260617,0.290831,0.003743,-0.123094,NaN,0.476795


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2518441 entries, 0 to 2518440
Data columns (total 31 columns):
 #   Column                Dtype  
---  ------                -----  
 0   offer_depersanalised  int64  
 1   goods_depersanalised  int64  
 2   sum_length            int64  
 3   attrs+title_score     float64
 4   offer_price           float64
 5   goods_price           float64
 6   goods_category_id     float64
 7   target                int64  
 8   id                    object 
 9   b                     float64
 10  c                     float64
 11  d                     float64
 12  e                     float64
 13  f                     float64
 14  g                     float64
 15  i                     float64
 16  j                     float64
 17  k                     float64
 18  l                     float64
 19  n                     float64
 20  o                     float64
 21  p                     float64
 22  q                     float64
 23  r      

In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363835 entries, 0 to 363834
Data columns (total 30 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   offer_depersanalised  363835 non-null  int64  
 1   goods_depersanalised  363835 non-null  int64  
 2   sum_length            363835 non-null  int64  
 3   attrs+title_score     363835 non-null  float64
 4   offer_price           363835 non-null  float64
 5   goods_price           304864 non-null  float64
 6   goods_category_id     363704 non-null  float64
 7   id                    363835 non-null  object 
 8   b                     95265 non-null   float64
 9   c                     95265 non-null   float64
 10  d                     95265 non-null   float64
 11  e                     95265 non-null   float64
 12  f                     95265 non-null   float64
 13  g                     95265 non-null   float64
 14  i                     363835 non-null  float64
 15  

Выделим признаки для обучения и целевой признак.

In [8]:
features_train = train.drop('target', axis=1)
target_train = train['target']

In [9]:
features_test = test

Выделим категориальный и числовые признаки.

In [10]:
columns = train.columns
display(columns)

Index(['offer_depersanalised', 'goods_depersanalised', 'sum_length',
       'attrs+title_score', 'offer_price', 'goods_price', 'goods_category_id',
       'target', 'id', 'b', 'c', 'd', 'e', 'f', 'g', 'i', 'j', 'k', 'l', 'n',
       'o', 'p', 'q', 'r', 's', 'v', 'w', 'x', 'y', 'dist', 'dist_title'],
      dtype='object')

In [11]:
cat_columns = ['offer_depersanalised','goods_depersanalised','goods_category_id','id']

In [12]:
num_columns = [col for col in list(train.columns) if col not in cat_columns and col != 'target']
display(num_columns)

['sum_length',
 'attrs+title_score',
 'offer_price',
 'goods_price',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'i',
 'j',
 'k',
 'l',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 'v',
 'w',
 'x',
 'y',
 'dist',
 'dist_title']

Масштабируем числовые признаки.

In [13]:
scaler = RobustScaler()
scaler.fit(features_train[num_columns])

features_train[num_columns] = scaler.transform(features_train[num_columns])
features_test[num_columns] = scaler.transform(features_test[num_columns])

In [14]:
features_train.head()

,offer_depersanalised,goods_depersanalised,sum_length,attrs+title_score,offer_price,goods_price,goods_category_id,id,b,c,...,p,q,r,s,v,w,x,y,dist,dist_title
0,295140,1396793,-0.815972,0.514477,-0.212756,NaN,14.0,295140$1396793,NaN,NaN,...,0.952109,0.792532,0.299591,0.477861,0.740229,0.631890,-1.784412,-0.860365,NaN,0.195954
1,65291,1396586,-0.814236,0.952959,-0.257521,NaN,14.0,65291$1396586,NaN,NaN,...,NaN,NaN,NaN,NaN,3.030042,0.242926,-1.879233,0.080249,NaN,1.314102
2,39232,1396244,-0.814236,1.651293,-0.240794,NaN,14.0,39232$1396244,NaN,NaN,...,NaN,NaN,NaN,NaN,0.911007,-0.399422,-0.838434,0.014964,NaN,0.802196
3,39232,1396513,-0.814236,1.651293,-0.240794,NaN,14.0,39232$1396513,NaN,NaN,...,NaN,NaN,NaN,NaN,0.911007,-0.399422,-0.838434,0.014964,NaN,0.025758
4,65052,1396237,-0.814236,1.509082,-0.210951,NaN,14.0,65052$1396237,NaN,NaN,...,NaN,NaN,NaN,NaN,0.546885,0.576816,-1.750162,-0.736988,NaN,-0.657750


In [15]:
features_test.head()

,offer_depersanalised,goods_depersanalised,sum_length,attrs+title_score,offer_price,goods_price,goods_category_id,id,b,c,...,p,q,r,s,v,w,x,y,dist,dist_title
0,64819,1396468,-0.814236,0.888213,-0.297232,NaN,14.0,64819$1396468,NaN,NaN,...,NaN,NaN,NaN,NaN,0.153716,-0.514422,1.134826,-0.220021,NaN,-0.263506
1,64819,1396235,-0.814236,0.888213,-0.297232,NaN,14.0,64819$1396235,NaN,NaN,...,NaN,NaN,NaN,NaN,0.153716,-0.514422,1.134826,-0.220021,NaN,-0.283128
2,64819,1396318,-0.814236,0.888213,-0.297232,NaN,14.0,64819$1396318,NaN,NaN,...,NaN,NaN,NaN,NaN,0.153716,-0.514422,1.134826,-0.220021,NaN,-0.679648
3,359959,1396281,-0.810764,1.138526,-0.265223,NaN,14.0,359959$1396281,-0.015285,0.680696,...,0.050086,0.483774,-0.567569,0.011622,1.849057,-1.667112,0.298883,1.278973,-0.040162,1.079110
4,142700,717657,-0.810764,0.004958,1.454392,5.137583,2.0,142700$717657,NaN,NaN,...,-0.308173,1.062073,-1.497069,1.651525,2.013236,2.817970,0.051214,-1.166291,NaN,2.016829


Избавимся от пропусков в категории товаров, поставим заглушку.

In [16]:
train['goods_category_id'].isna().sum()

833

In [17]:
test['goods_category_id'].isna().sum()

131

In [18]:
features_train['goods_category_id'].fillna(-999, inplace=True)
features_test['goods_category_id'].fillna(-999, inplace=True)

/tmp/ipykernel_34/768752844.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  features_train['goods_category_id'].fillna(-999, inplace=True)
/tmp/ipykernel_34/768752844.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True

Заменим тип данных в категории товаров.

In [19]:
features_train['goods_category_id'] = features_train['goods_category_id'].astype('str')

In [20]:
features_test['goods_category_id'] = features_test['goods_category_id'].astype('str')

# Машинное обучение.
Применим модель *CatBoostClassifier*. 

In [21]:
model_cb = CatBoostClassifier(random_state=784, task_type = 'GPU', learning_rate = 0.1, verbose = 0)

In [22]:
%%time
model_cb.fit(features_train, target_train, cat_features = cat_columns)

CPU times: user 7min 50s, sys: 19.9 s, total: 8min 10s
Wall time: 3min 38s


Передадим тестовую выборку в модель и выполним предсказание.

In [23]:
prediction = model_cb.predict(features_test) 
display(prediction)

array([0, 0, 0, ..., 0, 0, 0])

Превратим массив в датафрейм.

In [24]:
df_prediction= pd.DataFrame(prediction, columns = ['target'])

Загрузим пример сабмита предсказания.

In [25]:
sample_submission = pd.read_csv('/kaggle/input/sample-submission/sample_submission.csv')

Удалим исходный столбец с предсказаниями.

In [26]:
sample_submission= sample_submission.drop('target', axis=1)

Присоединим наши предсказания.

In [27]:
sample_submission = pd.concat([sample_submission, df_prediction], axis=1)

Создадим датасет для сабмита.

In [28]:
sample_submission.to_csv('sample_submission.csv', index= False)

In [29]:
sample_submission = pd.read_csv('/kaggle/working/sample_submission.csv')

In [30]:
sample_submission

,id,target
0,64819$1396468,0
1,64819$1396235,0
2,64819$1396318,0
3,359959$1396281,0
4,142700$717657,0
...,...,...
363830,122775$310950,0
363831,419632$342465,0
363832,369393$130129,0
363833,24514$130142,0


Наше решение позволило добиться значения метрики F1 = 0,73.